In [1]:
import torch
from transformers import SegformerFeatureExtractor
import sys
sys.path.insert(1, '../utils/')
import os

/home/emir/anaconda3/envs/emirenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from create_dataloaders import EddyDatasetTrain, EddyDatasetValid

In [3]:
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
feature_extractor.reduce_labels = False
feature_extractor.size = 128

/home/emir/anaconda3/envs/emirenv/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [4]:
input_image_dir = "/home/emir/dev/segmentation_eddies/downloads/data4test/aug_data/"
mask_image_dir = "/home/emir/dev/segmentation_eddies/downloads/data4test/aug_label/"
val_image_dir = "/home/emir/dev/segmentation_eddies/downloads/data4test/data/"
val_mask_dir = "/home/emir/dev/segmentation_eddies/downloads/data4test/label/"

In [5]:
train_len = len(os.listdir(input_image_dir))
split = int(0.85 * train_len)

In [6]:
train_data = EddyDatasetTrain(feature_extractor=feature_extractor, input_image_dir=input_image_dir, mask_image_dir=mask_image_dir, split=split)
valid_data = EddyDatasetValid(feature_extractor=feature_extractor, input_image_dir=val_image_dir, mask_image_dir=val_mask_dir, split=split)